### Install dependencies

In [1]:
!pip install boto3 sagemaker langchain langchain-community langchain-core faiss-cpu requests opensearch-py sentence-transformers langchain-text-splitters requests-aws4auth qdrant-client -U


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install --upgrade --quiet  lark qdrant-client


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


### Load CSV data from S3

In [3]:
!pwd

/Users/robertgreer/Documents/Education/Berkeley/Classes/Datasci210_Capstone/Enterprise-RAG/notebooks


In [4]:
import boto3
import pandas as pd

s3 = boto3.client('s3')
bucket_name = 'recipes-rag'

In [5]:
file_key = 'recipes_w_cleaning_time_combined_features.parquet'
s3.download_file(bucket_name, file_key, f'../data/{file_key}')
df = pd.read_parquet(f'../data/{file_key}')

df.head()

,Name,RecipeCategory,Description,Keywords_string,RecipeIngredientQuantities,RecipeIngredientParts,RecipeInstructions,AggregatedRating,ReviewCount,CookTime_Minutes,PrepTime_Minutes,TotalTime_Minutes,Combined_Features,Combined_Features_Clean
0,Low-Fat Berry Blue Frozen Dessert,Frozen Desserts,Make and share this Low-Fat Berry Blue Frozen ...,Dessert Low Protein Low Cholesterol Healthy Fr...,"[4, 1⁄4, 1, 1]","[blueberries, granulated sugar, vanilla yogurt...",Toss 2 cups berries with sugar. Let stand for ...,4.5,4.0,1440,45,1485,Low-Fat Berry Blue Frozen Dessert Frozen Desse...,Low-Fat Berry Blue Frozen Dessert Frozen Desse...
1,Biryani,Chicken Breast,Make and share this Biryani recipe from Food.com.,Chicken Thigh & Leg Chicken Poultry Meat Asian...,"[1, 4, 2, 2, 8, 1⁄4, 8, 1⁄2, 1, 1, 1⁄4, 1⁄4, 1...","[saffron, milk, hot green chili peppers, onion...",Soak saffron in warm milk for 5 minutes and pu...,3.0,1.0,25,240,265,Biryani Chicken Breast Make and share this Bir...,Biryani Chicken Breast Make share Biryani reci...
2,Best Lemonade,Beverages,This is from one of my first Good House Keepi...,Low Protein Low Cholesterol Healthy Summer < 6...,"[1 1⁄2, 1, None, 1 1⁄2, None, 3⁄4]","[sugar, lemons, rind of, lemon, zest of, fresh...","Into a 1 quart Jar with tight fitting lid, put...",4.5,10.0,5,30,35,Best Lemonade Beverages This is from one of my...,Best Lemonade Beverages one first Good House K...
3,Carina's Tofu-Vegetable Kebabs,Soy/Tofu,This dish is best prepared a day in advance to...,Beans Vegetable Low Cholesterol Weeknight Broi...,"[12, 1, 2, 1, 10, 1, 3, 2, 2, 2, 1, 2, 1⁄2, 1⁄...","[extra firm tofu, eggplant, zucchini, mushroom...","Drain the tofu, carefully squeezing out excess...",4.5,2.0,20,1440,1460,Carina's Tofu-Vegetable Kebabs Soy/Tofu This d...,Carina's Tofu-Vegetable Kebabs Soy/Tofu dish b...
4,Cabbage Soup,Vegetable,Make and share this Cabbage Soup recipe from F...,Low Protein Vegan Low Cholesterol Healthy Wint...,"[46, 4, 1, 2, 1]","[plain tomato juice, cabbage, onion, carrots, ...",Mix everything together and bring to a boil. R...,4.5,11.0,30,20,50,Cabbage Soup Vegetable Make and share this Cab...,Cabbage Soup Vegetable Make share Cabbage Soup...


In [6]:
df.columns

Index(['Name', 'RecipeCategory', 'Description', 'Keywords_string',
       'RecipeIngredientQuantities', 'RecipeIngredientParts',
       'RecipeInstructions', 'AggregatedRating', 'ReviewCount',
       'CookTime_Minutes', 'PrepTime_Minutes', 'TotalTime_Minutes',
       'Combined_Features', 'Combined_Features_Clean'],
      dtype='object')

In [7]:
str(df.iloc[0]['Combined_Features_Clean'])

"Low-Fat Berry Blue Frozen Dessert Frozen Desserts Make share Low-Fat Berry Blue Frozen Dessert recipe Food.com. Toss 2 cups berries sugar. Let stand 45 minutes, stirring occasionally. Transfer berry-sugar mixture food processor. Add yogurt process smooth. Strain fine sieve. Pour baking pan (or transfer ice cream maker process according manufacturers' directions). Freeze uncovered edges solid centre soft. Transfer processor blend smooth again. Return pan freeze edges solid. Transfer processor blend smooth again. Fold remaining 2 cups blueberries. Pour plastic mold freeze overnight. Let soften slightly serve. Dessert Low Protein Low Cholesterol Healthy Free Of... Summer Weeknight Freezer Easy"

In [8]:
df = df[df['AggregatedRating'].notna()]
df.shape

(269294, 14)

### Load data into chunked documents

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

In [10]:
embedding_model = HuggingFaceEmbeddings(model_name="multi-qa-mpnet-base-dot-v1")

/Users/robertgreer/Documents/Education/Berkeley/Classes/Datasci210_Capstone/Enterprise-RAG/venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/robertgreer/Documents/Education/Berkeley/Classes/Datasci210_Capstone/Enterprise-RAG/venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


### Note: chunking may not be useful for us as the EDA has the following about description length
print(new_recipe['Combined_Features_Clean'].str.len().describe())


count    522517.000000\
mean        211.461774\
std         122.388646\
min          44.000000\
25%         131.000000\
50%         180.000000\
75%         255.000000\
max        4174.000000\
Name: Combined_Features_Clean, dtype: float64

In [11]:
def create_textsplitter(chunks, overlaps):
    splits = {}
    for chunk in chunks:
        for overlap in overlaps:
            splits[f'chunk{str(chunk)}_overlap{str(overlap)}'] = RecursiveCharacterTextSplitter(chunk_size=chunk, chunk_overlap=overlap)
    return splits

In [12]:
chunks = [128, 256, 512, 1024, 2048]
overlaps = [.25, .2, .15, .1]

text_splits = create_textsplitter(chunks, overlaps)

In [13]:
text_splits['chunk1024_overlap0.25']

In [14]:
# def create_documents(df):
#     documents = []
#     for index, row in df.iterrows():
#         metadata = {
#             # 'recipe_id': str(row['RecipeId']) if not pd.isna(row['RecipeId']) else 'No ID Available',
#             'name': str(row['Name']) if not pd.isna(row['Name']) else 'No Name Available',
#             # 'cook_time': str(row['CookTime']) if not pd.isna(row['CookTime']) else 'No Cook Time Available',
#             # 'prep_time': str(row['PrepTime']) if not pd.isna(row['PrepTime']) else 'No Prep Time Available',
#             # 'total_time': str(row['TotalTime']) if not pd.isna(row['TotalTime']) else 'No Total Time Available',
#             'recipe_category': str(row['RecipeCategory']) if not pd.isna(row['RecipeCategory']) else 'No Category Available',
#             # 'keywords': str(row['Keywords']) if not pd.isna(row['Keywords']).all() else 'No Keywords Available',
#             'aggregated_rating': str(row['AggregatedRating']) if not pd.isna(row['AggregatedRating']) else 'No Rating Available',
#             'review_count': str(row['ReviewCount']) if not pd.isna(row['ReviewCount']) else 'No Reviews Available',
#             # 'calories': str(row['Calories']) if not pd.isna(row['Calories']) else 'No Calories Information Available',
#             # 'fat_content': str(row['FatContent']) if not pd.isna(row['FatContent']) else 'No Fat Content Available',
#             # 'saturated_fat_content': str(row['SaturatedFatContent']) if not pd.isna(row['SaturatedFatContent']) else 'No Saturated Fat Content Available',
#             # 'cholesterol_content': str(row['CholesterolContent']) if not pd.isna(row['CholesterolContent']) else 'No Cholesterol Content Available',
#             # 'sodium_content': str(row['SodiumContent']) if not pd.isna(row['SodiumContent']) else 'No Sodium Content Available',
#             # 'carbohydrate_content': str(row['CarbohydrateContent']) if not pd.isna(row['CarbohydrateContent']) else 'No Carbohydrate Content Available',
#             # 'sugar_content': str(row['SugarContent']) if not pd.isna(row['SugarContent']) else 'No Sugar Content Available',
#             # 'protein_content': str(row['ProteinContent']) if not pd.isna(row['ProteinContent']) else 'No Protein Content Available',
#             # 'recipe_servings': str(row['RecipeServings']) if not pd.isna(row['RecipeServings']) else 'No Servings Information Available',
#             # 'recipe_yield': str(row['RecipeYield']) if not pd.isna(row['RecipeYield']) else 'No Yield Information Available'
#         }

#         # Use Combined_Features_Clean for the document content
#         text = str(row['Combined_Features_Clean'])
#         doc = Document(page_content=text, metadata=metadata)
#         documents.append(doc)
        
#     return documents

def create_documents(df):
    df_copy = df.copy(deep=True)
    df = df.dropna(subset=["AggregatedRating"])
    df_copy = df_copy.fillna("")  # Convert NA values to empty strings
    df_copy = df_copy.astype(str)  # Cast all columns to string

    documents = []
    for _index, row in df_copy.iterrows():
        metadata = {
            "name": row["Name"] if row["Name"] else "No Name Available",
            "description": (
                row["Description"] if row["Description"] else "No Description Available"
            ),
            "recipe_category": (
                row["RecipeCategory"]
                if row["RecipeCategory"]
                else "No Category Available"
            ),
            "keywords": (
                row["Keywords_string"]
                if row["Keywords_string"]
                else "No Keywords Available"
            ),
            "recipe_ingredient_parts": (
                row["RecipeIngredientParts"]
                if row["RecipeIngredientParts"]
                else "No Recipe Ingredient Parts Available"
            ),
            "recipe_instructions": (
                row["RecipeInstructions"]
                if row["RecipeInstructions"]
                else "No Recipe Instructions Available"
            ),
            "aggregated_rating": (
                row["AggregatedRating"]
                if row["AggregatedRating"]
                else "No Rating Available"
            ),
            "review_count": (
                row["ReviewCount"] if row["ReviewCount"] else "No Reviews Available"
            ),
        }

        # List of fields to be included in the document content
        content_field = (
            row["Combined_Features"]
            if row["Combined_Features"]
            else "No Content Available"
        )

        # Create the document content using the combined features field
        doc = Document(page_content=content_field, metadata=metadata)
        documents.append(doc)

    return documents


In [15]:
# Take a sample of 100K receipes 
documents = create_documents(df.sample(n=100))

In [16]:
documents[0]

Document(metadata={'name': 'Passion Fruit Colada', 'description': 'Make and share this Passion Fruit Colada recipe from Food.com.', 'recipe_category': 'Beverages', 'keywords': 'Fruit Caribbean Vegan < 15 Mins Beginner Cook Easy', 'recipe_ingredient_parts': "['passion fruit juice' 'cream of coconut' 'rum']", 'recipe_instructions': 'Combine ice, passion fruit juice, cream of coconut and rum, if using, in a blender and blend until smooth.', 'aggregated_rating': '5.0', 'review_count': '4.0'}, page_content='Passion Fruit Colada Beverages Make and share this Passion Fruit Colada recipe from Food.com. Combine ice, passion fruit juice, cream of coconut and rum, if using, in a blender and blend until smooth. Fruit Caribbean Vegan < 15 Mins Beginner Cook Easy')

In [17]:
len(documents)

100

In [18]:
def split_documents_with_metadata(documents, text_splitter):
    split_docs = []
    for doc in documents:
        chunks = text_splitter.split_text(doc.page_content)
        for i, chunk in enumerate(chunks):
            split_docs.append(Document(page_content=chunk, metadata={**doc.metadata, "chunk_id": i}))
    return split_docs

In [19]:
split_documents = {key: split_documents_with_metadata(documents, text_splitter_recursive) for key, text_splitter_recursive in text_splits.items()}

In [20]:
split_documents['chunk256_overlap0.25'][0]

Document(metadata={'name': 'Passion Fruit Colada', 'description': 'Make and share this Passion Fruit Colada recipe from Food.com.', 'recipe_category': 'Beverages', 'keywords': 'Fruit Caribbean Vegan < 15 Mins Beginner Cook Easy', 'recipe_ingredient_parts': "['passion fruit juice' 'cream of coconut' 'rum']", 'recipe_instructions': 'Combine ice, passion fruit juice, cream of coconut and rum, if using, in a blender and blend until smooth.', 'aggregated_rating': '5.0', 'review_count': '4.0', 'chunk_id': 0}, page_content='Passion Fruit Colada Beverages Make and share this Passion Fruit Colada recipe from Food.com. Combine ice, passion fruit juice, cream of coconut and rum, if using, in a blender and blend until smooth. Fruit Caribbean Vegan < 15 Mins Beginner Cook Easy')

In [21]:
# We have > 500,000 recipes, this takes a long time to run
from langchain_community.vectorstores import Qdrant

# From documents with no chunking
qdrant_store = Qdrant.from_documents(documents,
    embedding_model,
    location=":memory:",
)

# from split documents (specify the chunk and overlap key)
# qdrant_store = Qdrant.from_documents(split_documents['chunk1024_overlap0.25'],
#     embedding_model,
#     location=":memory:",
# )

## Create llm query call

In [22]:
import json

In [23]:
bedrock_client = boto3.client('bedrock-runtime', region_name="us-east-1")

In [24]:
baseline_sys_prompt = """
You are a helpful assistant and expert in cooking recipes.

Before answering, always make at least one call to query_food_recipe_vector_db
to retrieve the relevant context of recipes and ingredients to generate an informed
and high-quality response to the user prompt but NEVER exceed a MAXIMUM of 
3 calls to the query_food_recipe_vector_db function.

Provide a response to the user prompt about food with recommended recipes and instructions.
"""

In [25]:
recipe_db_query_tool = {
  "name": "query_food_recipe_vector_db",
  "description": """
      Queries the vector database containing food recipes to retrieve the most relevant documents. 
      This function allows the model to generate and execute multiple queries as necessary to gather comprehensive context, 
      such as ingredients, preparation steps, and metadata like cuisine and diet type, ensuring accurate and thorough responses to user queries.
      """,
  "input_schema": {
    "type": "object",
    "properties": {
      "queries": {
        "type": "array",
        "items": {
          "type": "string",
          "description": "A query generated by the model to run against the vector database to fetch recipe documents."
        },
        "description": "A list of queries generated by the model to run against the vector database to fetch recipe documents."
      }
    },
    "required": ["queries"]
  }
}


In [26]:
MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"

def query_bedrock_llm(messages):
    response = bedrock_client.invoke_model(
        modelId=MODEL_ID,
        body=json.dumps({
            'anthropic_version': 'bedrock-2023-05-31', # This is required to use chat style messages object 
            'system': baseline_sys_prompt,
            'messages': messages,
            'max_tokens': 3000,
            "tools": [recipe_db_query_tool],

            # This config forces the model to always call the recipe db query tool atleast once 
            # https://docs.anthropic.com/en/docs/build-with-claude/tool-use#controlling-claudes-output
            # "tool_choice": {
            #     "type": "tool",
            #     "name": recipe_db_query_tool['name']
            # },
            
            # TODO: TUNE THESE VALUES
            'temperature': 0.1, 
            'top_p': 0.9
        })
    )
    response_body = json.loads(response.get('body').read())
    
    return response_body

## Create Retrieval Chainfrom langchain_core.prompts import ChatPromptTemplate

In [27]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableMap

In [28]:
def format_docs(docs):
    formatted_docs = []
    for doc in docs:
        formatted_docs.append(f"Metadata: {doc.metadata}\n")
    content = "\n\n".join(formatted_docs)
    
    return content

In [29]:
baseline_user_prompt = """
### Here is the context:
{context}

### Here is a user prompt:
{query}
"""

In [30]:
def process_prompt_basic(query_args):
    prompt_with_context = baseline_user_prompt.replace("{context}", query_args['context'])
    prompt_with_query = prompt_with_context.replace("{query}", query_args['query'])
    
    # This format doesn't matter much now, but we will use it later to 
    # persist chat history for continuous dialogue
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt_with_query
                }
            ]
        }
    ]
    
    return messages

## Create Initial Retriever

In [31]:
qdrant_retriever_rerank = qdrant_store.as_retriever(search_type='mmr', search_kwargs={"k": 5, 'lambda_mult': 0.5})


### Self Query

In [32]:
!pip install langchain_openai
!pip install python-dotenv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [33]:
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv, find_dotenv


In [34]:
load_dotenv()


True

In [54]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# OPENAI_API_KEY 

In [36]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

metadata_field_info = [
    AttributeInfo(
        name="name",
        description="The name of the recipe",
        type="string",
    ),
    AttributeInfo(
        name="description",
        description="A brief description of the recipe",
        type="string",
    ),
    AttributeInfo(
        name="recipe_category",
        description="The category of the recipe, such as 'Quick Breads', 'Desserts', etc.",
        type="string",
    ),
    AttributeInfo(
        name="keywords",
        description="Keywords associated with the recipe",
        type="string",
    ),
    AttributeInfo(
        name="recipe_ingredient_parts",
        description="The ingredients required for the recipe",
        type="string",
    ),
    AttributeInfo(
        name="recipe_instructions",
        description="The instructions to prepare the recipe",
        type="string",
    ),
    AttributeInfo(
        name="aggregated_rating",
        description="The aggregated rating for the recipe",
        type="string",
    ),
    AttributeInfo(
        name="review_count",
        description="The number of reviews for the recipe",
        type="string",
    ),
]
document_content_description = "Detailed information about a recipe"

llm = ChatOpenAI(
    # model='gpt-4-turbo',
    model='gpt-4o',
    temperature=0,
    openai_api_key=OPENAI_API_KEY,
    
)


In [37]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

reranker_model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")
cross_encoder = CrossEncoderReranker(model=reranker_model, top_n=1)
# compression_retriever = ContextualCompressionRetriever(
#     base_compressor=compressor, base_retriever=qa_retriever
# )


### Full chain

In [38]:
def filtered_qdrant_store(documents=[]):
    filtered_qdrant_store = Qdrant.from_documents(documents,
        embedding_model,
        location=":memory:",
    )
    return filtered_qdrant_store

In [39]:
def qa_message_prompt(user_prompt):
    f"""
system: You are a helpful assistant and expert in cooking recipes.

You are evaluating for consistency of the user query.
Please retrieve documents that DO NOT violate dietary restrictions, allergies, 
or any requirements dictated by the user.

user: {user_prompt}
"""

In [40]:
def self_query_retriever(documents=[]):
    return SelfQueryRetriever.from_llm(
        llm, filtered_qdrant_store(documents), document_content_description, metadata_field_info, verbose=True
    )

In [41]:
def qa_retriever_wrapper(dict):
    documents = dict.pop('documents')
    query = dict.pop('query')
    qa_retriever = SelfQueryRetriever.from_llm(
        llm, filtered_qdrant_store(documents), document_content_description, metadata_field_info, verbose=True
    )
    return {"documents": qa_retriever.invoke(qa_message_prompt(query)),
            "query": query}

In [42]:
def cross_encoder_wrapper(dict):
    documents = dict.pop('documents')
    query = dict.pop('query')
    # print(f'documents: {documents}')
    # print(f'query: {query}')
    # return {"documents": cross_encoder.compress_documents(query=query, documents=documents),
    #         "query": query}
    return cross_encoder.compress_documents(query=query, documents=documents)

In [43]:
qdrant_rag_chain_rerank = (
    RunnableMap(
        {"documents": qdrant_retriever_rerank,
         "query": RunnablePassthrough()}
    ) 
    | qa_retriever_wrapper
    | cross_encoder_wrapper
    # | format_docs
    # | process_prompt_basic
)

In [44]:
results = qdrant_rag_chain_rerank.invoke("Give me a peanut free Thai dish")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [45]:
type(results)

list

In [46]:
results

[Document(metadata={'name': 'Chicken Chow Mein - Easy!', 'description': "From BBC's Chinese Food Made Easy, the best recent TV cookery show. Very simple to make, and tasty! I added a little more five-spice than the original recipe. Omit the chilli sauce if you wish, or add a little more to give it extra zing.\r\nIf you don't have groundnut oil (peanut oil), use vegetable oil, NOT olive oil, which is too strong flavoured.\r\nServes two as a main course, or four as a side dish.", 'recipe_category': 'One Dish Meal', 'keywords': 'Chicken Breast Chicken Poultry Meat Chinese Asian Spicy Savory < 15 Mins Beginner Cook Stir Fry Easy Inexpensive', 'recipe_ingredient_parts': "['chicken breasts' 'dark soy sauce' 'five-spice powder' 'chili sauce'\n 'cornflour' 'red bell pepper' 'bean sprouts' 'spring onion'\n 'light soy sauce' 'fresh ground black pepper']", 'recipe_instructions': 'Cook the noodles in boiling water until al dente. Drain, rinse under cold water and drain again. Drizzle with a little

## Dynamic Function Calls

In [47]:
def generate_message(prompt):
    if type(prompt) != str:
        raise ValueError(f'Tried to call message generate_message with non-string input: {prompt}')

    return {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": prompt
            }
        ]
    }

In [48]:
def generate_tool_message(fn_results):
    return {
        "role": "user",
        "content": fn_results
    }

In [49]:
def message_handler(existing_chat_history, prompt, is_tool_message=False):
    # Fn results is an array of tool response objects
    # message structure needs to reflect that
    if is_tool_message:
        user_message = generate_tool_message(prompt)
    else:
        user_message = generate_message(prompt)
    existing_chat_history.append(user_message)

    # Parse the response content
    response_body = query_bedrock_llm(existing_chat_history)
    llm_message = {
        'role': response_body['role'],
        'content': response_body['content']
    }

    # Add the response message to the chat history
    existing_chat_history.append(llm_message)
    
    return [response_body, llm_message, existing_chat_history]

In [50]:
# Executes a list of queries and returns a list of document results
def handle_vector_db_queries(queries, retriever=qdrant_rag_chain_rerank): 
    context_docs = []
    for query in queries:
        query_results = retriever.invoke(query)
        context_docs.extend(query_results)

    return context_docs

In [51]:
# Takes as an argument to LLM message content, returns a list of the fn result objects
def handle_function_calls(tool_call_message_content, retriever=qdrant_rag_chain_rerank):
    tool_results = []
    
    for tool_call in tool_call_message_content:
        # Only process messages from the LLM that are function calls
        if tool_call['type'] != 'tool_use':
            continue
        fn_id = tool_call['id']
        fn_name = tool_call['name']
        fn_args = tool_call['input']
        fn_result = {
            "type": "tool_result",
            "tool_use_id": fn_id,
        }   

        if fn_name == 'query_food_recipe_vector_db':
            if 'queries' not in fn_args:
                print(f"ERROR: Tried to call {fn_name} with invalid args {fn_args}, skipping..")
                fn_result['content'] = ""
                fn_result['is_error'] = True
                tool_results.append(fn_result)
                continue
                
            print(f"Model called {fn_name} with args {fn_args}")
            context_docs = handle_vector_db_queries(fn_args['queries'], retriever)
            context_str = format_docs(context_docs)
            fn_result['content'] = context_str
            tool_results.append(fn_result)
            
        # TODO: handle web search invocation here
        else:
            print(f"ERROR: Attempted call to unknown function {fn_name}")
            fn_result['content'] = ""
            fn_result['is_error'] = True
            tool_results.append(fn_result)

    return tool_results

In [52]:
def run_chat_loop(prompt, retriever=qdrant_rag_chain_rerank):
    print(f"[User]: {prompt}")
    response_body, llm_message, chat_history = message_handler(existing_chat_history=[], prompt=prompt)
    
    
    # The model wants to call tools, call them, provide response, repeat until content is generated
    while response_body['stop_reason'] == 'tool_use':
        fn_results = handle_function_calls(tool_call_message_content=llm_message['content'], retriever=retriever)

        # Send function results back to LLM as a new message with the existing chat history
        response_body, llm_message, chat_history = message_handler(
            existing_chat_history=chat_history, 
            prompt=fn_results,
            is_tool_message=True
        )

    # The model is done calling tools
    print(f"\n[Model]: {llm_message['content'][0]['text']}")
    return f"\n[Model]: {llm_message['content'][0]['text']}"

In [53]:
run_chat_loop("Give me a peanut free Thai dish")

[User]: Give me a peanut free Thai dish
Model called query_food_recipe_vector_db with args {'queries': ['thai recipes without peanuts']}
Model called query_food_recipe_vector_db with args {'queries': ['thai curry recipes without peanuts']}
Model called query_food_recipe_vector_db with args {'queries': ['thai vegetable curry without peanuts']}

[Model]: <search_quality_reflection>
The search for "thai vegetable curry without peanuts" did not return any new relevant results beyond the Thai beef curry recipe from the previous search. While that recipe provides a good peanut-free Thai option, having a vegetarian Thai curry would give the user more variety to choose from. However, with the 3 query limit reached, I will provide my recommendation based on the results obtained.
</search_quality_reflection>

<search_quality_score>4</search_quality_score>

<result>
Here are some delicious peanut-free Thai dish recommendations for you:

Thai Red Curry Beef (Crock Pot)
Ingredients:
- Beef steak, c

'\n[Model]: <search_quality_reflection>\nThe search for "thai vegetable curry without peanuts" did not return any new relevant results beyond the Thai beef curry recipe from the previous search. While that recipe provides a good peanut-free Thai option, having a vegetarian Thai curry would give the user more variety to choose from. However, with the 3 query limit reached, I will provide my recommendation based on the results obtained.\n</search_quality_reflection>\n\n<search_quality_score>4</search_quality_score>\n\n<result>\nHere are some delicious peanut-free Thai dish recommendations for you:\n\nThai Red Curry Beef (Crock Pot)\nIngredients:\n- Beef steak, cut into chunks \n- Flour and salt for coating\n- Red bell pepper, sliced\n- Onion, sliced\n- Coconut milk\n- Red curry paste\n- Lime juice\n- Fish sauce\n- Soy sauce\n\nInstructions:\n1. Toss the beef chunks in a mixture of flour and salt to coat.\n2. Place the coated beef, sliced bell pepper, and sliced onion in a slow cooker spr